In [50]:
import torch
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50
from PGD import PGD
    
cuda = True if torch.cuda.is_available() else False
print(cuda)
norm = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

preprocess = transforms.Compose([
    transforms.Resize((224,224)), # Resize with 224
    transforms.ToTensor(), # making the matrix shape to [C,H,W] and range of [0,1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalizing with ImageNet mean and std)
])


val_imagefolder = ImageFolder(root='/workspace/my_data/imagenet/val', transform=preprocess)
val_dataloader = DataLoader(val_imagefolder, batch_size=1, shuffle=False)

# TAIG_imagefolder = ImageFolder(root='/workspace/my_data/TAIG_test', transform=preprocess)
# TAIG_dataloader = DataLoader(TAIG_imagefolder, batch_size=1, shuffle=False)


model = models.resnet50(pretrained=True)
model.eval() # batch가 1개일 때 가능하게함 Batchnorm, Dropout 사용안함
# Construct the CAM object once, and then re-use it on many images:
attack = PGD(model, cuda=cuda)
cos = nn.CosineSimilarity(dim=2)

max_list = []
min_list = []

for ori_imgs, ori_labels in val_dataloader:

    ori_imgs = ori_imgs.cuda()
    pgd_imgs = attack(ori_imgs, ori_labels)
    # pgd_input_tensor = attack(ori_imgs, ori_labels)
    # Note: input_tensor can be a batch tensor with several images!

    holder = []        
            
    def forwardhooking(m, inp, out):
        holder.append(out)
    # forwardhooking = ForwardHooking

    for name, m in model.named_modules():
        if name == 'layer4.2':
            m.register_forward_hook(forwardhooking)

    model(ori_imgs)
    model(pgd_imgs)
    
    # get cosine similarity along 2048channel in last layer
    sim = cos(holder[0][:,:,-1], holder[1][:,:,-1])
    max_ind = torch.argmax(sim).data.item()
    min_ind = torch.argmin(sim).data.item()

    max_list.append(max_ind)
    min_list.append(min_ind)
    

max_dict = {}
min_dict = {}
for i, j in zip(max_list, min_list):
    if i in max_dict:
        max_dict[i] += 1
    else:
        max_dict[i] = 1
        
    if j in min_dict:
        min_dict[j] += 1
    else:
        min_dict[j] = 1


min_dict_sorted = sorted(min_dict.items(), key=lambda x: x[1], reverse=True)
max_dict_sorted = sorted(max_dict.items(), key=lambda x: x[1], reverse=True)
torch.cuda.empty_cache()

True


1179m

In [51]:
min_dict_sorted

[(0, 6210),
 (3, 4422),
 (2, 4171),
 (6, 3750),
 (5, 3691),
 (7, 3387),
 (4, 3040),
 (1, 2480),
 (8, 2456),
 (13, 1930),
 (11, 1605),
 (9, 1472),
 (17, 1155),
 (10, 1048),
 (12, 1013),
 (15, 839),
 (16, 826),
 (22, 779),
 (14, 574),
 (18, 545),
 (19, 454),
 (23, 396),
 (21, 301),
 (26, 283),
 (29, 269),
 (25, 264),
 (20, 250),
 (28, 228),
 (24, 227),
 (27, 168),
 (35, 167),
 (32, 121),
 (31, 108),
 (33, 102),
 (30, 96),
 (36, 92),
 (39, 78),
 (40, 73),
 (43, 67),
 (34, 60),
 (37, 51),
 (38, 49),
 (49, 48),
 (46, 43),
 (54, 40),
 (44, 37),
 (45, 36),
 (53, 35),
 (55, 34),
 (60, 32),
 (50, 29),
 (48, 28),
 (41, 23),
 (52, 22),
 (56, 21),
 (42, 21),
 (58, 21),
 (64, 19),
 (57, 15),
 (47, 15),
 (63, 14),
 (67, 12),
 (68, 12),
 (51, 10),
 (70, 9),
 (72, 9),
 (59, 8),
 (66, 8),
 (62, 7),
 (73, 7),
 (69, 6),
 (78, 6),
 (65, 6),
 (76, 6),
 (61, 6),
 (79, 5),
 (71, 5),
 (83, 4),
 (75, 4),
 (77, 3),
 (90, 3),
 (80, 3),
 (89, 2),
 (95, 2),
 (102, 2),
 (74, 2),
 (87, 2),
 (111, 2),
 (92, 2),
 (85,

In [65]:
max_dict_sorted

[(1, 571),
 (0, 490),
 (20, 481),
 (10, 478),
 (2, 476),
 (12, 474),
 (14, 472),
 (19, 466),
 (4, 448),
 (8, 439),
 (6, 427),
 (3, 418),
 (15, 417),
 (5, 416),
 (9, 411),
 (11, 398),
 (18, 395),
 (27, 385),
 (34, 384),
 (21, 378),
 (24, 377),
 (41, 373),
 (7, 372),
 (48, 364),
 (42, 363),
 (16, 363),
 (25, 361),
 (47, 361),
 (30, 361),
 (51, 352),
 (17, 343),
 (33, 340),
 (31, 339),
 (23, 338),
 (13, 337),
 (32, 332),
 (26, 330),
 (37, 326),
 (46, 325),
 (36, 323),
 (61, 317),
 (28, 314),
 (38, 314),
 (45, 313),
 (39, 309),
 (52, 307),
 (29, 307),
 (22, 300),
 (59, 297),
 (44, 297),
 (65, 296),
 (66, 295),
 (58, 284),
 (43, 271),
 (40, 268),
 (53, 266),
 (49, 265),
 (54, 264),
 (57, 263),
 (50, 262),
 (62, 259),
 (84, 251),
 (56, 249),
 (87, 249),
 (60, 247),
 (81, 242),
 (74, 238),
 (55, 236),
 (82, 228),
 (35, 227),
 (68, 226),
 (76, 226),
 (85, 225),
 (64, 223),
 (86, 222),
 (88, 221),
 (94, 220),
 (77, 219),
 (113, 219),
 (71, 215),
 (75, 213),
 (72, 212),
 (100, 211),
 (73, 210),


In [62]:
len(max_dict_sorted)

896

In [63]:
len(min_dict_sorted)

106

## Consideration

1. Small number of channels make different results (dominated by small number of channels)

2. Number of maximum similarity is 896 however number of minimun similarity is 106.

======================================================================================================================================================================================================

## Select top k channels

In [9]:
import torch
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50
from PGD import PGD
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1,2"  # Set the GPUs 2 and 3 to use
    
cuda = True if torch.cuda.is_available() else False
print(cuda)
norm = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

preprocess = transforms.Compose([
    transforms.Resize((224,224)), # Resize with 224
    transforms.ToTensor(), # making the matrix shape to [C,H,W] and range of [0,1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalizing with ImageNet mean and std)
])


val_imagefolder = ImageFolder(root='/workspace/my_data/imagenet/val', transform=preprocess)
val_dataloader = DataLoader(val_imagefolder, batch_size=1, shuffle=False)

# TAIG_imagefolder = ImageFolder(root='/workspace/my_data/TAIG_test', transform=preprocess)
# TAIG_dataloader = DataLoader(TAIG_imagefolder, batch_size=1, shuffle=False)


model = models.resnet50(pretrained=True)
model.eval() # batch가 1개일 때 가능하게함 Batchnorm, Dropout 사용안함
# Construct the CAM object once, and then re-use it on many images:
attack = PGD(model, cuda=cuda)
cos = nn.CosineSimilarity(dim=2)

max_list = []
min_list = []

iter = 0
for ori_imgs, ori_labels in val_dataloader:

    ori_imgs = ori_imgs.cuda()
    pgd_imgs = attack(ori_imgs, ori_labels)
    # pgd_input_tensor = attack(ori_imgs, ori_labels)
    # Note: input_tensor can be a batch tensor with several images!

    holder = []        
            
    def forwardhooking(m, inp, out):
        holder.append(out)
    # forwardhooking = ForwardHooking

    for name, m in model.named_modules():
        if name == 'layer4.2':
            m.register_forward_hook(forwardhooking)

    model(ori_imgs)
    model(pgd_imgs)
    
    # get cosine similarity along 2048channel in last layer
    sim = cos(holder[0][:,:,-1], holder[1][:,:,-1])
    _, max_indcies = torch.topk(sim, 5, dim=1)
    max_indcies = max_indcies.squeeze(0).tolist()
    _, min_indcies = torch.topk(sim, 5, dim=1, largest=False)
    min_indcies = min_indcies.squeeze(0).tolist()
    # max_ind = torch.argmax(sim).data.item()
    # min_ind = torch.argmin(sim).data.item()

    max_list.extend(max_indcies)
    min_list.extend(min_indcies)
    
    iter += 1
    if iter == 1000:
        break

max_dict = {}
min_dict = {}
for i, j in zip(max_list, min_list):
    if i in max_dict:
        max_dict[i] += 1
    else:
        max_dict[i] = 1
        
    if j in min_dict:
        min_dict[j] += 1
    else:
        min_dict[j] = 1


min_dict_sorted = sorted(min_dict.items(), key=lambda x: x[1], reverse=True)
max_dict_sorted = sorted(max_dict.items(), key=lambda x: x[1], reverse=True)
torch.cuda.empty_cache()

True


In [12]:
print(len(min_dict_sorted))
min_dict_sorted

134


[(13, 225),
 (8, 217),
 (17, 212),
 (5, 181),
 (7, 163),
 (3, 162),
 (2, 160),
 (11, 160),
 (6, 156),
 (16, 146),
 (4, 143),
 (0, 138),
 (22, 130),
 (29, 116),
 (9, 113),
 (28, 112),
 (26, 110),
 (14, 99),
 (15, 98),
 (25, 94),
 (23, 93),
 (27, 89),
 (18, 87),
 (19, 84),
 (10, 76),
 (21, 75),
 (24, 75),
 (40, 74),
 (12, 66),
 (31, 61),
 (36, 57),
 (39, 55),
 (38, 53),
 (35, 51),
 (30, 50),
 (20, 50),
 (32, 49),
 (50, 48),
 (1, 46),
 (45, 41),
 (53, 41),
 (54, 40),
 (37, 34),
 (49, 33),
 (55, 33),
 (33, 33),
 (56, 26),
 (60, 25),
 (41, 24),
 (34, 23),
 (42, 23),
 (48, 22),
 (46, 21),
 (64, 20),
 (57, 19),
 (43, 19),
 (70, 17),
 (58, 17),
 (44, 16),
 (52, 15),
 (62, 13),
 (81, 13),
 (63, 12),
 (72, 12),
 (68, 11),
 (71, 10),
 (51, 10),
 (80, 9),
 (73, 8),
 (69, 8),
 (67, 8),
 (79, 8),
 (74, 8),
 (97, 7),
 (75, 6),
 (66, 6),
 (47, 6),
 (77, 5),
 (106, 5),
 (61, 5),
 (65, 5),
 (83, 5),
 (78, 5),
 (76, 4),
 (89, 4),
 (128, 4),
 (107, 4),
 (91, 4),
 (82, 4),
 (85, 4),
 (109, 4),
 (90, 3),
 (

In [13]:
print(len(max_dict_sorted))
max_dict_sorted

975


[(189, 20),
 (94, 18),
 (135, 18),
 (41, 17),
 (103, 17),
 (166, 16),
 (59, 16),
 (227, 16),
 (81, 16),
 (492, 16),
 (117, 16),
 (47, 16),
 (181, 15),
 (149, 15),
 (153, 15),
 (118, 15),
 (61, 14),
 (445, 14),
 (128, 14),
 (307, 14),
 (131, 14),
 (206, 14),
 (159, 14),
 (342, 14),
 (65, 14),
 (287, 14),
 (82, 13),
 (99, 13),
 (113, 13),
 (384, 13),
 (195, 13),
 (279, 13),
 (250, 13),
 (155, 13),
 (299, 13),
 (156, 13),
 (129, 13),
 (258, 13),
 (49, 13),
 (88, 12),
 (77, 12),
 (127, 12),
 (48, 12),
 (185, 12),
 (58, 12),
 (5, 12),
 (108, 12),
 (157, 12),
 (333, 12),
 (19, 12),
 (340, 12),
 (140, 12),
 (124, 12),
 (240, 12),
 (261, 12),
 (95, 12),
 (397, 12),
 (323, 12),
 (74, 12),
 (312, 12),
 (244, 12),
 (213, 12),
 (217, 11),
 (373, 11),
 (355, 11),
 (281, 11),
 (42, 11),
 (34, 11),
 (193, 11),
 (377, 11),
 (285, 11),
 (231, 11),
 (220, 11),
 (10, 11),
 (50, 11),
 (45, 11),
 (215, 11),
 (290, 11),
 (464, 11),
 (375, 11),
 (168, 11),
 (32, 11),
 (348, 11),
 (114, 11),
 (137, 11),
 (73,